In [1]:
%pip install pandas
%pip install torch
%pip install numpy
%pip install sklearn
%pip install scikit-learn
%pip install pulearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

Note: you may need to restart the kernel to use updated packages.
  Using cached pulearn-0.0.11-py3-none-any.whl.metadata (11 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pulearn-0.0.11-py3-none-any.whl (18 kB)
Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl (11.0 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.


In [2]:
# import torch
# import torch.optim as optim
# import torch.nn as nn
import pandas as pd
import numpy as np
# from torch.utils.data import Dataset, TensorDataset, DataLoader
# from torch.utils.data.dataset import random_split
# import torch.nn.functional as F
# from pulearn import ElkanotoPuClassifier
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import precision_recall_fscore_support
# from pulearn import BaggingPuClassifier


In [ ]:
import pandas as pd

dat = pd.read_csv("PNWTFDB_2018_0_503points.csv")

dropped = [
    "GlobalID",
    "Date of Failure",
    "Other",
    "CreationDate",
    "Creator",
    "EditDate",
    "Editor",
    "Care Factors",
    "x",
    "y"
]
dat = dat.drop(columns=dropped, axis=1)

fill_none = [
    "Site Factors",
    "Root Failure",
    "Stem Failure",
    "Branch Failure",
    "Location and Percentage of Decay",
    "Weather factors",
    "Decay Present"
]

dat["Type of Soil"] = dat["Type of Soil"].fillna(value="Dirt")
dat.loc[:, fill_none] = dat[fill_none].fillna("None")

dat['Length of Failed Part'] = dat.groupby(['Failed Part', 'Tree Species'])['Length of Failed Part'].transform( lambda x: x.fillna(x.mean()))
dat['Length of Failed Part'] = dat.groupby(['Failed Part'])['Length of Failed Part'].transform( lambda x: x.fillna(x.mean()))
dat.to_csv('treedata.csv', index=False)

#print(dat["Length of Failed Part"])

def tree_mapping(df, name):
    t_map = {name: i for i, name in enumerate(df[name].unique(), start=1)}
    df[name] = df[name].map(t_map)

tree_mapping(dat, 'Tree Species')
tree_mapping(dat, 'Condition')
tree_mapping(dat, 'Site Factors')
tree_mapping(dat, 'Type of Soil')
tree_mapping(dat, 'Weather factors')
tree_mapping(dat, 'Failed Part')
tree_mapping(dat, 'Root Failure')
tree_mapping(dat, 'Branch Failure')
tree_mapping(dat, 'Stem Failure')
tree_mapping(dat, 'Location and Percentage of Decay')
tree_mapping(dat, 'Decay Present')
print(dat['Tree Species'])





0       1
1       2
2       3
3       3
4       4
       ..
498    45
499     1
500    45
501     3
502    75
Name: Tree Species, Length: 503, dtype: int64


In [5]:
svc = SVC(C=10, kernel='rbf', gamma=0.4, probability=True, random_state=42)

pu_estimator = ElkanotoPuClassifier(estimator=svc, hold_out_ratio=0.2)

y = np.concatenate([np.ones(400), np.zeros(103)])
#np.random.shuffle(y)
X = dat.values

#X, y = X[:len(y)], y

pu_estimator.fit(X, y)

ElkanotoPuClassifier(estimator=SVC(C=10, gamma=0.4, probability=True,
                                   random_state=42),
                     hold_out_ratio=0.2)

In [6]:
print(pu_estimator.predict(X))
print(pu_estimator.predict(X).shape)
print("\nComparison of estimator and PUAdapter(estimator):")
print(
    "Number of disagreements: {}".format(
        len(np.where(pu_estimator.predict(X) != np.ones(503))[0])
    )
)
print(
    "Number of agreements: {}".format(
        len(np.where(pu_estimator.predict(X) == np.ones(503))[0])
    )
)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [13]:
svc = SVC(C=1, kernel='rbf', gamma=0.4, probability=True, random_state=42)
pu_estimator = BaggingPuClassifier(
        estimator=svc, n_estimators=50)
pu_probability = BaggingPuClassifier(estimator=svc, n_estimators=250)
clf = pu_estimator.fit(X, y)
#print(pu_estimator.predict(X))
np.set_printoptions(threshold=np.inf)
print(np.array(pu_estimator.predict_proba(X)))
print(
    "Number of disagreements: {}".format(
        len(np.where(pu_estimator.predict(X) != np.ones(503))[0])
    )
)
print(
    "Number of agreements: {}".format(
        len(np.where(pu_estimator.predict(X) == np.ones(503))[0])
    )
)


[[0.11845446 0.88154554]
 [0.11845972 0.88154028]
 [0.11845915 0.88154085]
 [0.1184646  0.8815354 ]
 [0.11846441 0.88153559]
 [0.11846277 0.88153723]
 [0.1184629  0.8815371 ]
 [0.11846722 0.88153278]
 [0.11846522 0.88153478]
 [0.11846537 0.88153463]
 [0.11846582 0.88153418]
 [0.11847065 0.88152935]
 [0.11846989 0.88153011]
 [0.11845657 0.88154343]
 [0.11846176 0.88153824]
 [0.11845678 0.88154322]
 [0.11846356 0.88153644]
 [0.11846357 0.88153643]
 [0.11844785 0.88155215]
 [0.11845011 0.88154989]
 [0.11846124 0.88153876]
 [0.11846957 0.88153043]
 [0.11847068 0.88152932]
 [0.11848061 0.88151939]
 [0.11847931 0.88152069]
 [0.11847195 0.88152805]
 [0.11847944 0.88152056]
 [0.11849802 0.88150198]
 [0.11850154 0.88149846]
 [0.1184814  0.8815186 ]
 [0.11848301 0.88151699]
 [0.11850729 0.88149271]
 [0.11847254 0.88152746]
 [0.11848758 0.88151242]
 [0.11847714 0.88152286]
 [0.11847147 0.88152853]
 [0.11848098 0.88151902]
 [0.11847562 0.88152438]
 [0.118477   0.881523  ]
 [0.11847793 0.88152207]


In [8]:
from sklearn.svm import OneClassSVM
clf = OneClassSVM(kernel="rbf",gamma=3).fit(X)
print(clf.predict(X))
print(len(np.where(clf.predict(X) == np.ones(503))[0]))
print(clf.score_samples(X))


[ 1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 -1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1